In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
from skimage.data import imread
import os,cv2
from keras.models import Sequential, Model, load_model
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/bufoodphone/train.csv')
test = pd.read_csv('../input/bufoodphone/test.csv')
train.head(5)



In [ ]:
food_model = load_model(filepath='../input/foodphone-model1/model.h5')

In [ ]:
#food_model.summary()

## Randomly select 75 pics out of 750 pics from 101 classes (different random state )

### Validate  with 10% data

### Because we use cross-validation, we don't have pure validation dataset.

In [ ]:
y = train.food_type

In [ ]:
#100 / 750

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train.index, train.food_type, test_size=0.1, random_state=33, stratify=train.food_type)
#x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.1, random_state=1, stratify=y_val)

In [ ]:
print(x_val)

In [ ]:
X = train.iloc[x_val]

In [ ]:
print(X.describe())

In [ ]:
print(y_val.head())

## Resize all the validation images

In [ ]:
from tqdm import tqdm
train_dir = '../input/bufoodphone/train_images/train_images'

images = []
train_types = []

train_files = [train_dir + '/' + filename for filename in X.filename]
print(len(train_files))

In [ ]:
images = []
train_types = []
for filename in tqdm(train_files):
    if filename.endswith('jpg'):
        try:
            images.append(cv2.resize(cv2.imread(filename), (299,299), interpolation=cv2.INTER_CUBIC))
            train_types.append(train[train_dir + '/' + train['filename'] == filename]['food_type'])
        except Exception as e:
            print(str(e))



In [ ]:
images = np.array(images)


In [ ]:
print(images.shape)

In [ ]:
#train_types = np.array(train_types)
#train_types =train_types[:1000]
#print(len(train_types[1]))
#print(train_types[20][:1])
train_types_arr = np.concatenate(train_types)
#train_types_arr.reshape(train_types_arr.shape[0],1)
print(train_types_arr.shape)


In [ ]:
y_val = train_types_arr
print(y_val)
#print(y_test.shape)

In [ ]:
To_class = train.food_type.unique()
print(To_class)

## Use trained model to predict images (should be in the training data, the acc should be high)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from keras.optimizers import SGD, RMSprop, Adam
from keras.regularizers import l2

In [ ]:
x_val = images
print(x_val.shape)

In [ ]:
#food_model.summary()

In [ ]:
y_val_pred = food_model.predict(x_val)

In [ ]:
y_val_pred_num=np.argmax(y_val_pred, axis = 1)
print(y_val_pred_num)

In [ ]:
def num_to_class(index):
    return To_class[index]

y_val_pred_class = num_to_class(y_val_pred_num)
top_5_preds= np.argpartition(y_val_pred, -5)[:,-5:]
top_5_val_class = num_to_class(top_5_preds)
print(y_val_pred_class[0])
print(top_5_val_class[0])


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
cm = confusion_matrix(y_val_pred_class,y_val)

In [ ]:
%%time
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Compute confusion matrix
#cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

class_names = [To_class[i] for i in range(101)]

plt.figure()
fig = plt.gcf()
fig.set_size_inches(32, 32)
plot_confusion_matrix(cm, classes=class_names,
                      title='Confusion matrix, without normalization',
                      cmap=plt.cm.Blues)
plt.show()

In [ ]:
print(cm)

In [ ]:

print(accuracy_score(y_val_pred_class,y_val))

In [ ]:
cm_acc_dis = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#array([[1.        , 0.        , 0.        ],
#      [1.        , 0.        , 0.        ],
#      [0.        , 0.33333333, 0.66666667]])

#The diagonal entries are the accuracies of each class
acc_dis = cm_acc_dis.diagonal()

In [ ]:
print(acc_dis)

In [ ]:

import scipy.stats as stats
#plt.hist(acc_dis, bins=101)

density = stats.gaussian_kde(acc_dis)
n, x, _ = plt.hist(acc_dis, bins=101, 
                   histtype=u'bar', normed=True)  
plt.plot(x, density(x),color='red')
plt.title('Accuracy by Class histogram')
plt.show()



In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val_pred_class,y_val))


In [ ]:
import random
show_sample = random.sample(list(range(images.shape[0])),9)
print(show_sample )
_, ax = plt.subplots(3,3, figsize=(12,12))
for i in range(3):
    for j in range(3):
        ax[i,j].imshow(cv2.cvtColor(images[show_sample[i*3+j]], cv2.COLOR_BGR2RGB))
        ax[i,j].axis('on')
        #ax[i,j].set_title(train_types_arr[(i*200)+j], size = 20)
        #ax[i,j].set_title(train_types_arr[(i*200)+j], size = 20)
        
        match =y_val_pred_class[show_sample[i*3+j]]==  y_val[show_sample[i*3+j]]
        ec = (0.9, 0, .2)
        fc = (0.9, 0, .2)
        if match:
            ec = (0, .6, .1)
            fc = (0, .7, .2)
        # predicted label
        ax[i,j].text(5,5,"P: "+y_val_pred_class[show_sample[i*3+j]],size=15, rotation=0,
                ha="left", va="top",
                 bbox=dict(boxstyle="square",
                       ec=ec,
                       fc=fc,
                       )
                 )
        if not match:
            # true label
            ax[i,j].text(5,50,"A: "+y_val[show_sample[i*3+j]],size=15, rotation=0,
                ha="left", va="top",
                 bbox=dict(boxstyle="square",
                       ec=ec,
                       fc=fc,
                       )
                 )


In [ ]:
import operator

top_5_chances= np.partition(y_val_pred, -5)[:,-5:]
print(top_5_chances)

print(top_5_val_class)

In [ ]:
top_5_dict = {top_5_val_class[6187][i]:top_5_chances[6187][i] for i in range(5)}
sorted_top_5_dict = sorted(top_5_dict.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
print(sorted_top_5_dict)

In [ ]:
#print(top_5_preds[0])
#print(top_5_chances[0])
#print(top_5_dict)
#print(sorted_top_5_dict)
plt.imshow(cv2.cvtColor(images[6187], cv2.COLOR_BGR2RGB))
plt.axis('on')
print("%s: "%("Top-5 Predict"))

for i in range(5):
    print("%20s : %7.6f" %(str(sorted_top_5_dict[i][0]), sorted_top_5_dict[i][1]))